In [138]:
import requests
import pickle
import json

In [150]:
game_data = []
year = '2019'
season_type = '02' 
max_game_ID = 3

In [151]:
for i in range(0,max_game_ID):
    r = requests.get(url='http://statsapi.web.nhl.com/api/v1/game/'
        + year + season_type +str(i).zfill(4)+'/feed/live')
    data = r.json()
    game_data.append(data)

In [152]:
game_data = game_data[1:]

In [156]:
game_data

[{'copyright': 'NHL and the NHL Shield are registered trademarks of the National Hockey League. NHL and NHL team marks are the property of the NHL and its teams. © NHL 2022. All Rights Reserved.',
  'gamePk': 2019020001,
  'link': '/api/v1/game/2019020001/feed/live',
  'metaData': {'wait': 10, 'timeStamp': '20191106_204348'},
  'gameData': {'game': {'pk': 2019020001, 'season': '20192020', 'type': 'R'},
   'datetime': {'dateTime': '2019-10-02T23:00:00Z',
    'endDateTime': '2019-10-03T01:50:13Z'},
   'status': {'abstractGameState': 'Final',
    'codedGameState': '7',
    'detailedState': 'Final',
    'statusCode': '7',
    'startTimeTBD': False},
   'teams': {'away': {'id': 9,
     'name': 'Ottawa Senators',
     'link': '/api/v1/teams/9',
     'venue': {'id': 5031,
      'name': 'Canadian Tire Centre',
      'link': '/api/v1/venues/5031',
      'city': 'Ottawa',
      'timeZone': {'id': 'America/New_York', 'offset': -5, 'tz': 'EST'}},
     'abbreviation': 'OTT',
     'triCode': 'OTT',


In [178]:
len(game_data)

2

In [144]:
import pandas as pd
import json as js

In [180]:
players_df = pd.DataFrame()
for i in game_data:
    #print(i)
    tmp_players_df = pd.DataFrame(i['gameData']['players']).transpose()
    tmp_players_df = tmp_players_df.assign(gameId = i['gameData']['game']['pk'])
    tmp_players_df = tmp_players_df.assign(teamID = lambda x: x['currentTeam'].apply(lambda y: y['id']))
    tmp_players_df = tmp_players_df.assign(pPos = lambda x: x['primaryPosition'].apply(lambda y: y['name']))
    tmp_players_df = tmp_players_df.drop(columns=['currentTeam', 'primaryPosition'])
    players_df = players_df.append(tmp_players_df)
    
players_df

/var/folders/z6/nzpwwscs6qj7t6gldff9vtlw0000gn/T/ipykernel_8698/1380563361.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  players_df = players_df.append(tmp_players_df)
/var/folders/z6/nzpwwscs6qj7t6gldff9vtlw0000gn/T/ipykernel_8698/1380563361.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  players_df = players_df.append(tmp_players_df)


,id,fullName,link,firstName,lastName,primaryNumber,birthDate,currentAge,birthCity,birthStateProvince,...,weight,active,alternateCaptain,captain,rookie,shootsCatches,rosterStatus,gameId,teamID,pPos
ID8467950,8467950,Craig Anderson,/api/v1/people/8467950,Craig,Anderson,41,1981-05-21,41,Park Ridge,IL,...,186,True,False,False,False,L,Y,2019020001,9,Goalie
ID8480073,8480073,Erik Brannstrom,/api/v1/people/8480073,Erik,Brannstrom,26,1999-09-02,23,Eksjo,NaN,...,181,True,False,False,True,L,Y,2019020001,9,Defenseman
ID8477939,8477939,William Nylander,/api/v1/people/8477939,William,Nylander,88,1996-05-01,26,Calgary,AB,...,196,True,False,False,False,R,Y,2019020001,10,Center
ID8475718,8475718,Justin Holl,/api/v1/people/8475718,Justin,Holl,3,1992-01-30,30,Tonka Bay,MN,...,210,True,False,False,False,R,Y,2019020001,10,Defenseman
ID8476406,8476406,Nicholas Shore,/api/v1/people/8476406,Nicholas,Shore,26,1992-09-26,29,Denver,CO,...,198,True,False,False,False,R,Y,2019020001,10,Center
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID8474145,8474145,Robert Bortuzzo,/api/v1/people/8474145,Robert,Bortuzzo,41,1989-03-18,33,Thunder Bay,ON,...,216,True,False,False,False,R,Y,2019020002,19,Defenseman
ID8476880,8476880,Tom Wilson,/api/v1/people/8476880,Tom,Wilson,43,1994-03-29,28,Toronto,ON,...,220,True,False,False,False,R,Y,2019020002,15,Right Wing
ID8475098,8475098,Tyler Bozak,/api/v1/people/8475098,Tyler,Bozak,21,1986-03-19,36,Regina,SK,...,199,True,False,False,False,R,Y,2019020002,19,Center
ID8475170,8475170,Brayden Schenn,/api/v1/people/8475170,Brayden,Schenn,10,1991-08-22,31,Saskatoon,SK,...,200,True,False,False,False,L,Y,2019020002,19,Center


In [10]:
teams_df = pd.DataFrame(game_data['gameData']['teams']).transpose()
teams_df

,id,name,link,venue,abbreviation,triCode,teamName,locationName,firstYearOfPlay,division,conference,franchise,shortName,officialSiteUrl,franchiseId,active
away,9,Ottawa Senators,/api/v1/teams/9,"{'id': 5031, 'name': 'Canadian Tire Centre', '...",OTT,OTT,Senators,Ottawa,1992,"{'id': 17, 'name': 'Atlantic', 'nameShort': 'A...","{'id': 6, 'name': 'Eastern', 'link': '/api/v1/...","{'franchiseId': 30, 'teamName': 'Senators', 'l...",Ottawa,http://www.ottawasenators.com/,30,True
home,10,Toronto Maple Leafs,/api/v1/teams/10,"{'name': 'Scotiabank Arena', 'link': '/api/v1/...",TOR,TOR,Maple Leafs,Toronto,1926,"{'id': 17, 'name': 'Atlantic', 'nameShort': 'A...","{'id': 6, 'name': 'Eastern', 'link': '/api/v1/...","{'franchiseId': 5, 'teamName': 'Maple Leafs', ...",Toronto,http://www.mapleleafs.com/,5,True


In [11]:
games_df = pd.DataFrame(game_data['gameData']['game'],index=[0])
games_df = games_df.assign(startDate = game_data['gameData']['datetime']['dateTime'])
games_df = games_df.assign(endDate = game_data['gameData']['datetime']['endDateTime'])
games_df = games_df.assign(status = game_data['gameData']['status']['abstractGameState'])
games_df = games_df.assign(teamAwayID = game_data['gameData']['teams']['away']['id'])
games_df = games_df.assign(teamHomeID = game_data['gameData']['teams']['home']['id'])
games_df

,pk,season,type,startDate,endDate,status,teamAwayID,teamHomeID
0,2019020001,20192020,R,2019-10-02T23:00:00Z,2019-10-03T01:50:13Z,Final,9,10


In [83]:
events_df = pd.DataFrame(game_data['liveData']['plays']['allPlays'])
events_df = events_df.assign(eventId = lambda x: x['about'].apply(lambda y: y['eventIdx']))
events_df = events_df.assign(eventType = lambda x: x['result'].apply(lambda y: y['event']))
events_df = events_df.assign(eventTypeDesc = lambda x: x['result'].apply(lambda y: y['description']))
events_df = events_df.assign(period = lambda x: x['about'].apply(lambda y: y['period']))
events_df = events_df.assign(periodTime = lambda x: x['about'].apply(lambda y: y['periodTime']))
events_df = events_df.assign(periodType = lambda x: x['about'].apply(lambda y: y['periodType']))
events_df = events_df.assign(goalsHome = lambda x: x['about'].apply(lambda y: y['goals']['home']))
events_df = events_df.assign(goalsAway = lambda x: x['about'].apply(lambda y: y['goals']['away']))
events_df = events_df.assign(coX = lambda x: x['coordinates'].apply(lambda y: y['x'] if y else ''))
events_df = events_df.assign(coY = lambda x: x['coordinates'].apply(lambda y: y['y'] if y else ''))
#events_df = events_df.assign(playerId_1 = events_df['players'] if isinstance(events_df['players'],list) else '')
events_df = events_df.assign(gameId = game_data['gameData']['game']['pk'])

#result_players = dict()
#iterate over events
#for i in events_df['players']:        
 #   try:
  #      for idx, p in enumerate(i):
            #print(idx, '\n', p)
            
   #         result_players['player'+ str(idx)] = p['player']['id']

        #result_player1Id.append(i[0]['player']['id'])
    #except IndexError:
     #   pass
        #result_player1Id.append("")
    #except TypeError:
     #   pass
        #result_player1Id.append("")
        
    #print(result_players)
#events_df = events_df.assign(player1Id = result_player1Id)

#events_df

result_player1Id = []
result_player1Type = []

for i in events_df['players']:
    try:
        result_player1Id.append(i[0]['player']['id'])
        result_player1Type.append(i[0]['playerType'])
    except IndexError:
        result_player1Id.append("")
        result_player1Type.append("")
    except TypeError:
        result_player1Id.append('')
        result_player1Type.append("")
    except ValueError:
        result_player1Id.append('')
        result_player1Type.append("")     

result_player2Id = []
result_player2Type = []

for i in events_df['players']:
    try:
        result_player2Id.append(i[1]['player']['id'])
        result_player2Type.append(i[1]['playerType'])
    except IndexError:
        result_player2Id.append('')
        result_player2Type.append('')
    except TypeError:
        result_player2Id.append('')
        result_player2Type.append('')
    except ValueError:
        result_player2Id.append('')
        result_player2Type.append('')
        
result_player3Id = []
result_player3Type = []

for i in events_df['players']:
    try:
        result_player3Id.append(i[2]['player']['id'])
        result_player3Type.append(i[2]['playerType'])
    except IndexError:
        result_player3Id.append('')
        result_player3Type.append('')
    except TypeError:
        result_player3Id.append('')
        result_player3Type.append('')
    except ValueError:
        result_player3Id.append('')
        result_player3Type.append('')
        
result_player4Id = []
result_player4Type = []

for i in events_df['players']:
    try:
        result_player4Id.append(i[3]['player']['id'])
        result_player4Type.append(i[3]['playerType'])
    except IndexError:
        result_player4Id.append('')
        result_player4Type.append('')
    except TypeError:
        result_player4Id.append('')
        result_player4Type.append('')
    except ValueError:
        result_player4Id.append('')
        result_player4Type.append('')
        
events_df = events_df.assign(player1Id = result_player1Id)
events_df = events_df.assign(player1Type = result_player1Type)
events_df = events_df.assign(player2Id = result_player2Id)
events_df = events_df.assign(player2Type = result_player2Type)
events_df = events_df.assign(player3Id = result_player3Id)
events_df = events_df.assign(player3Type = result_player3Type)
events_df = events_df.assign(player4Id = result_player4Id)
events_df = events_df.assign(player4Type = result_player4Type)

events_df

,result,about,coordinates,players,team,eventId,eventType,eventTypeDesc,period,periodTime,...,coY,gameId,player1Id,player1Type,player2Id,player2Type,player3Id,player3Type,player4Id,player4Type
0,"{'event': 'Game Scheduled', 'eventCode': 'TOR1...","{'eventIdx': 0, 'eventId': 1, 'period': 1, 'pe...",{},NaN,NaN,0,Game Scheduled,Game Scheduled,1,00:00,...,,2019020001,,,,,,,,
1,"{'event': 'Period Ready', 'eventCode': 'TOR5',...","{'eventIdx': 1, 'eventId': 5, 'period': 1, 'pe...",{},NaN,NaN,1,Period Ready,Period Ready,1,00:00,...,,2019020001,,,,,,,,
2,"{'event': 'Period Start', 'eventCode': 'TOR8',...","{'eventIdx': 2, 'eventId': 8, 'period': 1, 'pe...",{},NaN,NaN,2,Period Start,Period Start,1,00:00,...,,2019020001,,,,,,,,
3,"{'event': 'Faceoff', 'eventCode': 'TOR9', 'eve...","{'eventIdx': 3, 'eventId': 9, 'period': 1, 'pe...","{'x': 0.0, 'y': 0.0}","[{'player': {'id': 8475166, 'fullName': 'John ...","{'id': 10, 'name': 'Toronto Maple Leafs', 'lin...",3,Faceoff,John Tavares faceoff won against Colin White,1,00:00,...,0.0,2019020001,8475166,Winner,8478400,Loser,,,,
4,"{'event': 'Goal', 'eventCode': 'TOR10', 'event...","{'eventIdx': 4, 'eventId': 10, 'period': 1, 'p...","{'x': 85.0, 'y': -1.0}","[{'player': {'id': 8480801, 'fullName': 'Brady...","{'id': 9, 'name': 'Ottawa Senators', 'link': '...",4,Goal,"Brady Tkachuk (1) Tip-In, assists: Connor Brow...",1,00:25,...,-1.0,2019020001,8480801,Scorer,8477015,Assist,8478400,Assist,8475883,Goalie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,"{'event': 'Missed Shot', 'eventCode': 'TOR839'...","{'eventIdx': 377, 'eventId': 839, 'period': 3,...","{'x': -39.0, 'y': -1.0}","[{'player': {'id': 8479318, 'fullName': 'Austo...","{'id': 10, 'name': 'Toronto Maple Leafs', 'lin...",377,Missed Shot,Auston Matthews Wide of Net,3,19:52,...,-1.0,2019020001,8479318,Shooter,,,,,,
378,"{'event': 'Period End', 'eventCode': 'TOR840',...","{'eventIdx': 378, 'eventId': 840, 'period': 3,...",{},NaN,NaN,378,Period End,End of 3rd Period,3,20:00,...,,2019020001,,,,,,,,
379,"{'event': 'Period Official', 'eventCode': 'TOR...","{'eventIdx': 379, 'eventId': 843, 'period': 3,...",{},NaN,NaN,379,Period Official,Period Official,3,20:00,...,,2019020001,,,,,,,,
380,"{'event': 'Game End', 'eventCode': 'TOR844', '...","{'eventIdx': 380, 'eventId': 844, 'period': 3,...",{},NaN,NaN,380,Game End,Game End,3,20:00,...,,2019020001,,,,,,,,


In [76]:
event_players_df = pd.DataFrame(game_data['liveData']['plays']['allPlays'])
event_players_df = event_players_df.assign(gameId = game_data['gameData']['game']['pk'])
event_players_df = event_players_df.assign(eventId = lambda x: x['about'].apply(lambda y: y['eventIdx']))
event_players_df = event_players_df.drop(columns=['result', 'about', 'coordinates', 'team'])
event_players_df = event_players_df.dropna()

result_player1Id = []
for i in event_players_df['players']:
    try:
        result_player1Id.append(i[1]['player']['id'])
    except IndexError:
        result_player1Id.append("")

event_players_df = event_players_df.assign(player1Id = result_player1Id)

result_player1Type = []
for i in event_players_df['players']:
    try:
        result_player1Type.append(i[1]['playerType'])
    except IndexError:
        result_player1Type.append("")
        
event_players_df = event_players_df.assign(player1Type = result_player1Type)

event_players_df


,players,gameId,eventId,player1Id,player1Type
3,"[{'player': {'id': 8475166, 'fullName': 'John ...",2019020001,3,8478400,Loser
4,"[{'player': {'id': 8480801, 'fullName': 'Brady...",2019020001,4,8477015,Assist
5,"[{'player': {'id': 8476919, 'fullName': 'Chris...",2019020001,5,8477939,Loser
6,"[{'player': {'id': 8479458, 'fullName': 'Nikit...",2019020001,6,,
8,"[{'player': {'id': 8473573, 'fullName': 'Artem...",2019020001,8,8477021,Loser
...,...,...,...,...,...
373,"[{'player': {'id': 8479318, 'fullName': 'Austo...",2019020001,373,8476919,Loser
374,"[{'player': {'id': 8474589, 'fullName': 'Tyler...",2019020001,374,8476879,Shooter
375,"[{'player': {'id': 8479318, 'fullName': 'Austo...",2019020001,375,,
376,"[{'player': {'id': 8478469, 'fullName': 'Thoma...",2019020001,376,8475883,Goalie


In [ ]:
result = []
for value in df["Marks"]:
    if value >= 33:
        result.append("Pass")
    elif value < 0 and value > 100:
        result.append("Invalid")
    else:
        result.append("Fail")
      
df["Result"] = result  
print(df)

In [126]:
df = pd.DataFrame(game_data['liveData']['plays']['allPlays'])
df = df.players.dropna()
for item in df:
    for p in item:
        print(p['player']['id'])
        print(p['playerType'])

8475166
Winner
8478400
Loser
8480801
Scorer
8477015
Assist
8478400
Assist
8475883
Goalie
8476919
Winner
8477939
Loser
8479458
Shooter
8473573
Winner
8477021
Loser
8474589
PenaltyOn
8479675
DrewBy
8475166
Winner
8476419
Loser
8478483
PlayerID
8476853
Shooter
8467950
Goalie
8476919
Winner
8475166
Loser
8474697
Blocker
8477341
Shooter
8478483
Shooter
8476331
Hitter
8477512
Hittee
8478857
Shooter
8476331
Shooter
8475883
Goalie
8474162
PlayerID
8476879
PlayerID
8476853
Shooter
8467950
Goalie
8477149
Hitter
8476879
Hittee
8473573
Winner
8477021
Loser
8479675
Shooter
8475197
Shooter
8467950
Goalie
8475197
Shooter
8467950
Goalie
8476331
Blocker
8477341
Shooter
8476879
Shooter
8467950
Goalie
8479318
PlayerID
8477341
Shooter
8467950
Goalie
8478400
PenaltyOn
8477939
DrewBy
8476419
Winner
8475166
Loser
8468493
PlayerID
8478483
Shooter
8467950
Goalie
8478483
Shooter
8475166
Winner
8476919
Loser
8479318
Shooter
8476919
Hitter
8478483
Hittee
8477939
Shooter
8467950
Goalie
8475197
Shooter
8467950
Goal

In [ ]:
for i in event_players_df['players']:
    if isinstance(i, list):
        print(type(i))
    else:
        print(type(i))

In [154]:
df_coo = pd.DataFrame(game_data['liveData']['plays']['allPlays'])
df_coo = df_coo['coordinates']
df_coo = df_coo.dropna()

for sor in df_coo:
    if sor:
        print(sor['x'])


0.0
85.0
0.0
28.0
69.0
-9.0
-69.0
-36.0
-32.0
-69.0
-74.0
-46.0
-96.0
-64.0
63.0
93.0
34.0
-59.0
75.0
20.0
-86.0
-42.0
-52.0
-62.0
-38.0
-87.0
-76.0
-81.0
-69.0
-94.0
-63.0
-66.0
-69.0
-70.0
-40.0
-78.0
-32.0
-80.0
-69.0
78.0
47.0
83.0
-69.0
-64.0
-38.0
4.0
83.0
-36.0
67.0
69.0
47.0
47.0
-97.0
-62.0
-97.0
96.0
69.0
18.0
56.0
-70.0
-69.0
-45.0
-98.0
50.0
69.0
-95.0
-34.0
-75.0
-96.0
-66.0
68.0
96.0
-69.0
-69.0
-38.0
-60.0
-41.0
-69.0
-62.0
20.0
70.0
62.0
0.0
-36.0
-95.0
-94.0
-42.0
-69.0
-96.0
-96.0
-93.0
67.0
69.0
43.0
69.0
-41.0
96.0
22.0
33.0
30.0
69.0
70.0
0.0
-43.0
-97.0
-96.0
81.0
-20.0
-36.0
-69.0
-31.0
-69.0
-69.0
11.0
75.0
56.0
86.0
0.0
-97.0
97.0
43.0
69.0
99.0
68.0
64.0
69.0
65.0
81.0
68.0
37.0
69.0
35.0
45.0
84.0
0.0
73.0
68.0
1.0
13.0
96.0
-83.0
0.0
12.0
9.0
33.0
-20.0
-76.0
83.0
0.0
0.0
-31.0
-40.0
-35.0
96.0
-43.0
-84.0
-37.0
79.0
95.0
73.0
-69.0
-75.0
-61.0
-82.0
-69.0
33.0
-20.0
96.0
69.0
-69.0
-60.0
-69.0
-85.0
-68.0
13.0
58.0
69.0
-94.0
-69.0
-69.0
-40.0
69.0
-89.0
69

In [14]:
df = pd.DataFrame(game_data['liveData']['plays']['allPlays'])
df = df['players']
for i in df:
    if isinstance(i, list):
        print(type(i))
    else:
        print(type(i))

<class 'float'>
<class 'float'>
<class 'float'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'float'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'float'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'float'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'float'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'float'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'float'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'float'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


In [195]:
df

0                                                    NaN
1                                                    NaN
2                                                    NaN
3      [{'player': {'id': 8475166, 'fullName': 'John ...
4      [{'player': {'id': 8480801, 'fullName': 'Brady...
                             ...                        
377    [{'player': {'id': 8479318, 'fullName': 'Austo...
378                                                  NaN
379                                                  NaN
380                                                  NaN
381                                                  NaN
Name: players, Length: 382, dtype: object